Вас пригласили на работу в коммерческую компанию, занимающуюся разработкой автоматизированных торговых агентов. Одной из первых ваших задач будет подготовка данных для дальнейшей обработки и построения модели. Пообщавшись с коллегами, вы узнали, что вам предстоит работать с несколькими типами активов: акциями из списка SnP500 и криптовалютами (BTC, ETH, SOL, XRP). Вам планируют поручить краткосрочную и среднесрочную торговлю.


Вам предлагается на основе предоставленной информации:


1. Создать git-репозиторий, где будет храниться исходный код вашего проекта. Если вы используете приватный репозиторий – дайте преподавателям курса доступ к нему, для возможности проверки ДЗ.
2. Добавить файл лицензии, который отражает ваш взгляд на конфиденциальность информации, которую вы подготовите в рамках данного курса.
3. Создать код на Python, который загрузит на ваш локальный компьютер данные о котировках ценных бумаг из списка SnP500 и котировки криптовалют (BTC, ETH, SOL, XRP).
4. Поскольку вам предстоит много работать с ними в дальнейшем, подготовьте автоматическое отображение графиков текущей ситуации.
5. Проверьте нет ли в данных пропусков или ошибок. Проанализируйте выбросы. Оцените, на самом ли деле это выбросы или реальные данные, с которыми предстоит работать.

In [ ]:
# Initialization cell

!pip install yfinance -qq
import yfinance as yf
import pandas_datareader as web
import pandas as pd
import numpy as np
import os
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import zscore


# Create Data directory if it doesn't exist
dir_stocks = os.path.join('data', 'stock')
if not os.path.exists(dir_stocks):
    os.makedirs(dir_stocks)
dir_crypto = os.path.join('data', 'crypto')
if not os.path.exists(dir_crypto):
    os.makedirs(dir_crypto)

In [ ]:
# Data download to subfolders cell

tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]['Symbol'].str.replace('.','-', regex=True).to_list()[:10]
data = yf.download(tickers=tickers,group_by='Ticker',multi_level_index=False,progress=False)
data_per_ticker = {}
for ticker in tickers:
    df = data[ticker]#.dropna()
    df.columns = [x.lower() for x in df.columns]
    df.index.name = df.index.name.lower()
    df.to_csv(os.path.join(dir_stocks, ticker + '.csv'))


# Download crypto data
cryptos = ['BTC-USD', 'ETH-USD', 'SOL-USD', 'XRP-USD']
for crypto in cryptos:
    data = yf.download(crypto, multi_level_index=False,progress=False)#.dropna()
    df.columns = [x.lower() for x in df.columns]
    df.index.name = df.index.name.lower()
    data.to_csv(os.path.join(dir_crypto, crypto.split("-")[0] + '.csv'))


In [ ]:
# Functions definition cell

def load_data(ticker: str, is_crypto: bool = False):
    df: pd.DataFrame = None
    path = dir_stocks
    if is_crypto:
        path = dir_crypto
    path = os.path.join(path, ticker + '.csv')
    if os.path.exists(path):
        df = pd.read_csv(path, parse_dates=['date'])
    else:
        print(f'No file for {ticker} at {path}')
    return df

def plot(df, time_start = None, time_end = None):
    if time_start is None:
        time_start = min(df['date'])
    if time_end is None:
        time_end = max(df['date'])
    df = df[(df['date'] >= time_start) & (df['date'] <= time_end)]
    fig = go.Figure(data=[go.Candlestick(x=df['date'],
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'])])
    fig.show()
    
def check_data_integrity(ticker, df):
    # Проверка общего количества пропусков
    missing_values = df.isnull().sum()
    # Проверка процентного соотношения пропусков
    missing_percentage = df.isnull().mean() * 100
    print(f'Процентное содержание пропусков для {ticker}: {missing_percentage}')
    print(missing_percentage)
    
def heatmap(ticker, df):
    # Heatmap пропусков
    sns.heatmap(df.isnull(), cbar=False)
    plt.show()
    # Bar plot пропусков
    missing_values.plot(kind='bar')
    plt.show()

def bounds(ticker, df, field):
    Q1 = df[field].quantile(0.25)
    Q3 = df[field].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return lower_bound, upper_bound

    
def quantile(ticker, df):
    def quantile_by_field(ticker, df, field):
        lower_bound, upper_bound = bounds(ticker, df, field)
        outliers = df[(df[field] < lower_bound) | (df[field] > upper_bound)]
        print(f'outliers for {ticker} {field}: {outliers}')
    for field in ['open', 'high', 'low', 'close']:
        quantile_by_field(ticker, df, field)
    
def z_score(ticker, df):
    def z_score_by_field(ticker, df, field):
        z_scores = np.abs(zscore(df[field]))
        outliers = df[z_scores > 3]
        print(f'ZScore for {ticker} {field}: {outliers}')
    for field in ['open', 'high', 'low', 'close']:
        z_score_by_field(ticker, df, field)
    
def clean_peaks(ticker, df):
    def clean_peaks_by_field(ticker, df, field):
        lower_bound, upper_bound = bounds(ticker, df, field)
        # Удаление выбросов
        df_cleaned = df[(df[field] >= lower_bound) & (df[field] <= upper_bound)]

        # Замена выбросов медианой
        median_value = df[field].median()
        df[field] = np.where((df[field] < lower_bound) | (df[field] > upper_bound), median_value, df[field])
    for field in ['open', 'high', 'low', 'close']:
        clean_peaks_by_field(ticker, df, field)


In [ ]:
ticker = 'A'
df = load_data(ticker)
time_start = pd.to_datetime('2000-01-01')
time_end   = pd.to_datetime('2000-01-05')
plot(df, time_start, time_end)

In [ ]:
check_data_integrity(ticker, df)

In [ ]:
quantile(ticker, df)

In [ ]:
z_score(ticker, df)

In [ ]:
clean_peaks(ticker, df)